In [1]:
import re
from copy import copy, deepcopy
from itertools import product

In [2]:
class DeterministicDie:
    
    def __init__(self, sides=100):
        self.value = 1
        self.sides = sides
        self.rolls = 0
    
    def roll(self):
        value = self.value
        self.value += 1
        if self.value > self.sides:
            self.value = 1
        self.rolls += 1
        return value

class QuantumDie:
    
    def roll(self):
        return [1, 2, 3]

In [3]:
regex = re.compile('Player ([0-9]+) starting position: ([0-9]+)')

class Player:
    
    def __init__(self, line):
        match = regex.match(line)
        self.number = int(match.group(1))
        self.pos = int(match.group(2))
        self.score = 0
    
    def move(self, times):
        self.pos = (self.pos + times - 1) % 10 + 1 
        self.score += self.pos
    
    def __repr__(self):
        return f'Player {self.number} [position={self.pos}, score {self.score}]'

In [4]:
def play(players):
    die = DeterministicDie(sides=100)
    while True:
        for i, player in enumerate(players):
            player.move(die.roll() + die.roll() + die.roll())
            if player.score >= 1000:
                return players[1-i].score * die.rolls

In [5]:
def play_parallel(players):
    memory = {}
    def rec(playing, other):
        if playing.score >= 21:
            return (1, 0)
        if other.score >= 21:
            return (0, 1)
        
        key = playing.pos, playing.score, other.pos, other.score
        if key in memory:
            return memory[key]
        
        counts = (0, 0)
        for d1, d2, d3 in product([1, 2, 3], repeat=3):
            pnew = copy(playing)
            pnew.move(d1 + d2 + d3)
            b, a = rec(other, pnew)
            counts = (counts[0] + a, counts[1] + b)
        
        memory[key] = counts
        return counts
    
    playing, other = players
    return rec(playing, other)

In [6]:
with open('input.txt', 'r') as f:
    players = [Player(l) for l in f.readlines()]

In [7]:
print(f'Part 1: {play(deepcopy(players))}')

Part 1: 513936


In [8]:
print(f'Part 2: {max(play_parallel(players))}')

Part 2: 105619718613031
